In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.max_colwidth', None) # show complete content of a cells in table
pd.set_option('display.max_columns', None)

# load data frame
# df = pd.read_excel(r"C:\Users\g21728\Desktop\Projekte\Glassdoor\Daten 2020\Fertige Dateien\1-1000.xlsx", sheet_name="Fertig")
# df = pd.read_excel(r"C:\Users\g21728\Desktop\Projekte\Glassdoor\Daten 2020\Fertige Dateien\1001-2000.xlsx", sheet_name="Fertig")
# df = pd.read_excel(r"C:\Users\g21728\Desktop\Projekte\Glassdoor\Daten 2020\Fertige Dateien\2001-3000.xlsx", sheet_name="Fertig")

# feather data file way faster than excel/csv
# df.to_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\glassdoor-data-cleaning\1_1000.ftr")
# df.to_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\glassdoor-data-cleaning\1001_2000.ftr")
# df.to_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\glassdoor-data-cleaning\2001_3000.ftr")

# df1 = pd.read_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\glassdoor-data-cleaning\1_1000.ftr")
# df2 = pd.read_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\glassdoor-data-cleaning\1001_2000.ftr")
df = pd.read_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\glassdoor-data-cleaning\2001_3000.ftr")

# frames = [df1, df2]

# df = pd.concat(frames, ignore_index=True)
df
# df.to_feather(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\glassdoor-data-cleaning\1_3000.ftr")

In [ ]:
df["employee_status"].unique()
filtered = df[df["date"].str.contains("GMT")]
filtered

In [ ]:
df.dropna(axis=0, how='all', inplace=True) # drop rows containing only empty values
# df.shape

# list(df.columns) # column list

# convert float columns to int
# df.dtypes # datatype of columns
float_columns = df.select_dtypes(include=np.number).columns.tolist()
df[float_columns] = df[float_columns].astype("Int64") #Int64 can store missing values as NaN

# convert time to year
df.insert(df.columns.get_loc("date")+1, "year", "") # insert new column after date column
df["year"] = df["date"].str[11:15]

# employee_status -> categories (0: former; 1: current)
# df["employee_status"].unique()
employee_status_categories = {'Current Employee': 1, 'Former Employee': 0} # dict with categories
df["employee_status"] = np.where(df['employee_status'].str.contains('former', case=False) == True, "Former Employee", "Current Employee") # if recommends column contains "outlook", then copy this value to new outlook column. else take value from outlook column
df["employee_status"] = df["employee_status"].replace(employee_status_categories).astype("Int64") # replace strings with ints
df.rename(columns={"employee_status": "current_employee"}, inplace=True) #inplace works with the current dataset. otherwise df["employee_status"] = ... is needed in front

# assign outlook categories
# print(df["positive_outlook"].unique()) #['Positive Outlook' nan 'Negative Outlook' 'Neutral Outlook']
positive_outlook_categories = {'Positive Outlook':2, "Negative Outlook":0, "Neutral Outlook":1}
df["positive_outlook"] = df["positive_outlook"].replace(positive_outlook_categories).astype("Int64")

# years at company -> tenure categories & full (1) vs part time (0)
# print(df["years_at_company"].unique()) # show unique values
df.insert(6, "full_time", "")
employee_status_categories = {True: 1, False: 0}
df["full_time"] = df["years_at_company"].str.contains("full-time").map(employee_status_categories).astype("Int64") # if string contains "full-time"

# tenure: cut "years at company" after "for"
df.insert(7, "tenure", "")
df["tenure"] = df["years_at_company"].str.split('for ').str[1]
# df["tenure"].unique()
# assign tenure categories
tenure_categories = {'more than 3 years':3, 'more than a year':2, 'more than 5 years':4,'more than 8 years':5, 'less than a year':1,'more than 10 years':6}
df["tenure"] = df["tenure"].replace(tenure_categories).astype("Int64")
# drop cleaned columns
# cols = list(df.columns.values) # create list of all column names
# if "date" in cols:
df.drop(columns=["date", "years_at_company"], inplace=True)

# correctly categorize erroneous "recommends" values
# print(df["recommends"].unique()) # -> ['Recommends' nan 'Approves of CEO' 'Positive Outlook' "Doesn't Recommend" 'Neutral Outlook' 'Negative Outlook' 'No opinion of CEO', 'Disapproves of CEO']
# df.insert(df.columns.get_loc("recommends")+1, "outlook_new", "") # insert new column after date column
df["positive_outlook"] = np.where(df['recommends'].str.contains('Outlook', case=False, regex=True) == True, df["recommends"], df["positive_outlook"]) # if recommends column contains "outlook", then copy this value to new outlook column. else take value from outlook column
# assign outlook categories
# print(df["positive_outlook"].unique()) #['Positive Outlook' nan 'Negative Outlook' 'Neutral Outlook']
positive_outlook_categories = {'Positive Outlook':2, "Negative Outlook":0, "Neutral Outlook":1}
df["positive_outlook"] = df["positive_outlook"].replace(positive_outlook_categories).astype("Int64")

# df.insert(df.columns.get_loc("recommends")+2, "approves_of_CEO_new", "") # insert new column after date column
df["approves_of_CEO"] = np.where(df['recommends'].str.contains('CEO', case=False, regex=True) == True, df["recommends"], df["approves_of_CEO"])
# assign ceo categories
# print(df["approves_of_CEO"].unique()) #['Approves of CEO' nan 'No opinion of CEO' 'Disapproves of CEO']
approves_of_CEO_categories = {'Approves of CEO':2, "Disapproves of CEO":0, "(?i)No opinion of CEO":1} #(?i) --> case insensitive 
df["approves_of_CEO"] = df["approves_of_CEO"].replace(approves_of_CEO_categories, regex=True).astype("Int64")


df["recommends"] = np.where(df['recommends'].str.contains('Outlook|CEO', case=False, regex=True) == True, np.NaN, df["recommends"]) # delete wrong values from recommend column
# assign recommends categories
# print(df["recommends"].unique()) #['Recommends' nan "Doesn't Recommend"]
recommends_categories = {'Recommends':1, "Doesn't Recommend":0}
df["recommends"] = df["recommends"].replace(recommends_categories).astype("Int64")

# split location to city & state
df.insert(df.columns.get_loc("location")+1, "state", "")
df.insert(df.columns.get_loc("location")+2, "city", "")
df["state"] = df["location"].str.split(',').str[1]
df["city"] = df["location"].str.split(',').str[0]
df.drop(columns=["location"], inplace=True)

df[:5]


In [ ]:
# vlookup (innerjoin) company_name and ticker
df_ticker = pd.read_excel(r"C:\Users\g21728\Desktop\Directors\Glassdoor Data Cleaning\glassdoor-data-cleaning\ticker_matching_russell3000_ISIN.xlsx", sheet_name="Sheet1")
df = pd.merge(df, df_ticker, on="company_name", how="inner")

# reorder columns
cols = list(df.columns.values) # create list of all column names
# print(cols)
first_columns = ['company_name', 'ISIN', 'ticker', 'year', 'rating_overall', 'rating_balance', 'rating_culture', 'rating_career', 'rating_comp', 'rating_mgmt', 'recommends', 'positive_outlook', 'approves_of_CEO', 'helpful', 'current_employee', 'full_time', 'tenure']
df = df[first_columns + [col for col in cols if col not in first_columns]]

# convert numpy NaN to panda <NA>, which is supposed to be better
df = df.replace(np.NaN, pd.NA)

df.head()

In [ ]:
# generate list of numeric columns to groupby
# print(df.select_dtypes(include=np.number).columns.tolist())
numeric_columns = df.select_dtypes(include=np.number).columns.tolist()

# groupby company and year based on mean
gk = df.groupby(by=["company_name", "ISIN", "ticker", "year"], as_index=False)[numeric_columns].mean().round(4) #as_index False -> show company_name in every cell
# gk = df.groupby(by=["company_name", "ticker", "year"])[numeric_columns].agg(['count','mean']).round(4).reset_index()

# number of observations per firm-year, based on rating_overall
count_column=df.groupby(["company_name", "year"])["rating_overall"].size()
# count_column

# merge mean and count dataframe
final = pd.merge(gk, count_column, on=["company_name", "year"], how="inner", suffixes=['', '_count'])
final.head()


In [ ]:
# export to csv
# final.dtypes

final.to_csv(r'C:\Users\g21728\Desktop\glassdoor_aggregated_ratings.csv', index=False, header=True, decimal=",", sep=';') #saves it with . as decimal seperator for whatever reason. stata can handle this, german excel cant. adjust seperator when importing csv via excel